将NHANES的饮食数据与FPED的数据关联，得出相应的食物的饮食模式，对应DGA中的饮食模式

In [29]:
import pandas as pd

In [30]:
# 导入数据
individual_firstday_df = pd.read_sas('e:/work/Paper/NHANES/0708Analysis Data/Diet/NHANES-Diet-data/DR1IFF_E.XPT')
individual_secondday_df = pd.read_sas('e:/work/Paper/NHANES/0708Analysis Data/Diet/NHANES-Diet-data/DR2IFF_E.XPT')

FPED_df = pd.read_excel('e:/work/Paper/NHANES/0708Analysis Data/Diet/FPED_0708.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [31]:
# 查看有多少编码可以匹配上，多少不能匹配上，以及匹配上的比例
# 全部匹配上
individual_day1_usda = individual_firstday_df['DR1IFDCD'].tolist()
individual_day2_usda = individual_secondday_df['DR2IFDCD'].tolist()

FOODCODE_in_FPED = FPED_df['FOODCODE'].to_list()

matched_counts = 0
unmatched_counts = 0 

for m in [individual_day1_usda, individual_day2_usda]:
    for q in individual_day1_usda:
        if q in FOODCODE_in_FPED:
            matched_counts += 1
        else:
            unmatched_counts += 1
    print('匹配成功:',matched_counts, '匹配不成功:',unmatched_counts, '比例:',matched_counts/(matched_counts+unmatched_counts))

匹配成功: 145703 匹配不成功: 0 比例: 1.0
匹配成功: 291406 匹配不成功: 0 比例: 1.0


In [32]:
# 合并两天数据
individual_concat_df = pd.concat([individual_firstday_df, individual_secondday_df])
individual_concat_df.reset_index(drop=True, inplace=True)
individual_concat_df.shape

(267044, 165)

In [33]:
#提取FPED中DGA中的相关变量
FPED_feature = [
    'FOODCODE',
    'F_TOTAL (cup eq.)', 'F_JUICE (cup eq.)',                                           #Fruits
    'V_TOTAL (cup eq.)',  'V_DRKGR (cup eq.)',                                          #Vegetables
    'PF_LEGUMES (oz. eq.)' ,                                                            #LEGUMES
    'G_WHOLE (oz. eq.)', 'G_REFINED (oz. eq.)',                                         #Grain
    'D_TOTAL (cup eq.)',                                                                #Dairy
    'PF_SEAFD_HI (oz. eq.)','PF_SEAFD_LOW (oz. eq.)', 'PF_TOTAL (oz. eq.)',             #Protein Foods
    'PF_NUTSDS (oz. eq.)', 'PF_SOY (oz. eq.)',                                      
    'ADD_SUGARS (tsp. eq.)'   
]

FPED_extract_df = FPED_df[FPED_feature]

In [34]:
# 提取饮食访谈中的食物编码及克重
individual_feature = [
    'SEQN',
    'DR1IFDCD', #USDA编码
    'DR2IFDCD', #USDA编码
    'DR1IGRMS', #克重
    'DR2IGRMS', #克重
]
# individual_extract_df 从合并两天数据中提取出的变量(individual_feature)
individual_extract_df = individual_concat_df[individual_feature]
individual_extract_df.fillna(0, inplace=True)   #填补缺失值，用于加和

individual_extract_df.loc[:,'DRIGRMS'] = individual_extract_df.loc[:, 'DR1IGRMS'] + individual_extract_df.loc[:, 'DR2IGRMS']    #合并克重为新变量DRIGRMS
individual_extract_df.loc[:,'DRIGRMS/100'] = individual_extract_df.loc[:,'DRIGRMS']/100 #由DRIGRMS除以100得出新的变量DRIGRMS/100 进行单位换算
individual_extract_df.loc[:,'DRIFDCD'] = individual_extract_df.loc[:, 'DR1IFDCD'] + individual_extract_df.loc[:, 'DR2IFDCD']    #和合并USDA编码为新变量DRIFDCD
individual_extract_df = individual_extract_df.loc[:, ['SEQN', 'DRIGRMS', 'DRIGRMS/100', 'DRIFDCD']]

C:\Users\Gstar\AppData\Local\Temp\ipykernel_156\591831136.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_extract_df.fillna(0, inplace=True)   #填补缺失值，用于加和
C:\Users\Gstar\AppData\Local\Temp\ipykernel_156\591831136.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_extract_df.loc[:,'DRIGRMS'] = individual_extract_df.loc[:, 'DR1IGRMS'] + individual_extract_df.loc[:, 'DR2IGRMS']    #合并克重为新变量DRIGRMS
C:\Users\Gstar\AppData\Local\Temp\ipykernel_156\591831136.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [35]:
#将每个食物的USDA编码与FPED中的编码匹配
individual_FPED_merge_df = pd.merge(individual_extract_df, FPED_extract_df, left_on='DRIFDCD', right_on='FOODCODE', how='left')
individual_FPED_merge_df = individual_FPED_merge_df.drop(labels=['DRIFDCD'], axis=1)
individual_FPED_merge_df

,SEQN,DRIGRMS,DRIGRMS/100,FOODCODE,F_TOTAL (cup eq.),F_JUICE (cup eq.),V_TOTAL (cup eq.),V_DRKGR (cup eq.),PF_LEGUMES (oz. eq.),G_WHOLE (oz. eq.),G_REFINED (oz. eq.),D_TOTAL (cup eq.),PF_SEAFD_HI (oz. eq.),PF_SEAFD_LOW (oz. eq.),PF_TOTAL (oz. eq.),PF_NUTSDS (oz. eq.),PF_SOY (oz. eq.),ADD_SUGARS (tsp. eq.)
0,41475.0,160.00,1.6000,64132010,0.4,0.4,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00
1,41475.0,710.40,7.1040,92101500,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00
2,41475.0,4.00,0.0400,91200040,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00
3,41475.0,10.08,0.1008,12120100,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.2,0.0,0.0,0.00,0.00,0.0,0.00
4,41475.0,63.62,0.6362,52215200,0.0,0.0,0.0,0.0,0.0,0.0,4.53,0.0,0.0,0.0,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320037,51623.0,96.00,0.9600,53203500,0.0,0.0,0.0,0.0,0.0,0.0,2.57,0.0,0.0,0.0,1.32,1.11,0.0,6.25
320038,51623.0,118.50,1.1850,94000100,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00
320039,51623.0,124.00,1.2400,61210250,0.4,0.4,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00
320040,51623.0,236.80,2.3680,92301060,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,1.19


In [36]:
# 构建食物对应DAG评分的组分 --  individual_DGA_standard_df  
individual_transformed_df = pd.DataFrame()          #individual_transformed_df  第一天和第二天合并数据经过FPED转换后的数据
individual_transformed_df['SEQN'] = individual_FPED_merge_df['SEQN']
individual_transformed_df['FOODCODE'] = individual_FPED_merge_df['FOODCODE']

FPED_feature = [
    'F_TOTAL (cup eq.)', 'F_JUICE (cup eq.)',                 #Fruits
    'V_TOTAL (cup eq.)',  'V_DRKGR (cup eq.)',                #Vegetables
    'PF_LEGUMES (oz. eq.)' ,                                  #LEGUMES
    'G_WHOLE (oz. eq.)', 'G_REFINED (oz. eq.)',               #Grain
    'D_TOTAL (cup eq.)',                                      #Dairy
    'PF_SEAFD_HI (oz. eq.)','PF_SEAFD_LOW (oz. eq.)',   'PF_TOTAL (oz. eq.)',         #Protein Foods
    'PF_NUTSDS (oz. eq.)', 'PF_SOY (oz. eq.)',
    'ADD_SUGARS (tsp. eq.)'
]

transformed_name = [
    'F_TOTAL', 'F_JUICE',                    #Fruits
    'V_TOTAL',  'V_DRKGR',                   #Vegetables
    'PF_LEGUMES' ,                           #LEGUMES 
    'G_WHOLE', 'G_REFINED',                  #Grain
    'D_TOTAL',                               #Dairy
    'PF_SEAFD_HI','PF_SEAFD_LOW', 'PF_TOTAL',           #Protein Foods
    'PF_NUTSDS', 'PF_SOY',
    'ADD_SUGARS'
    ]

for i,q in zip(transformed_name, FPED_feature):
    print(i)
    individual_transformed_df[i] = individual_FPED_merge_df['DRIGRMS/100']*individual_FPED_merge_df[q]



individual_DGA_standard_df = individual_transformed_df.groupby(by='SEQN').sum().drop('FOODCODE', axis=1)    
individual_DGA_standard_df = individual_DGA_standard_df/2             

F_TOTAL
F_JUICE
V_TOTAL
V_DRKGR
PF_LEGUMES
G_WHOLE
G_REFINED
D_TOTAL
PF_SEAFD_HI
PF_SEAFD_LOW
PF_TOTAL
PF_NUTSDS
PF_SOY
ADD_SUGARS


In [39]:
# 添加 第一天第二天总能量；饱和脂肪酸；单不饱和脂肪酸； 多不饱和脂肪酸； 糖分；
individual_firstday_total_df = pd.read_sas('E:/work/Paper/NHANES/0708Analysis Data/Diet/NHANES-Diet-data/DR1TOT_E.XPT')
individual_secondday_total_df = pd.read_sas('E:/work/Paper/NHANES/0708Analysis Data/Diet/NHANES-Diet-data/DR2TOT_E.XPT')

# 合并数据
individual_total_concat_df = pd.concat([individual_firstday_total_df, individual_secondday_total_df])
individual_total_concat_df.reset_index(drop=True, inplace=True)

In [40]:
# 构建参与者的  卡路里， 单不饱和脂肪酸， 多不饱和脂肪酸， 饱和脂肪酸， 钠 的摄入量    individual_total_df  每星期
# individual_total_df
individual_total_extract_features = [
    'SEQN',
    'DR1TKCAL', 'DR2TKCAL',     #卡路里
    'DR1TMFAT', 'DR2TMFAT',     #单不饱和脂肪酸
    'DR1TPFAT', 'DR2TPFAT',     #多不饱和脂肪酸
    'DR1TSFAT', 'DR2TSFAT',     #饱和脂肪酸'
    'DR1TSODI', 'DR2TSODI',     #钠
]

individual_total_merge_features = [
    'DRTKCAL',     #卡路里
    'DRTMFAT',     #单不饱和脂肪酸
    'DRTPFAT',     #多不饱和脂肪酸
    'DRTSFAT',     #饱和脂肪酸'
    'DRTSODI',     #钠
]
individual_total_extract_df = individual_total_concat_df[individual_total_extract_features]
individual_total_extract_df.fillna(0, inplace=True)

individual_total_df = pd.DataFrame()
individual_total_df['SEQN'] = individual_total_extract_df['SEQN']
individual_total_df['DRTKCAL'] = individual_total_extract_df['DR1TKCAL'] + individual_total_extract_df['DR2TKCAL']
individual_total_df['DRTMFAT'] = individual_total_extract_df['DR1TMFAT'] + individual_total_extract_df['DR2TMFAT']
individual_total_df['DRTPFAT'] = individual_total_extract_df['DR1TPFAT'] + individual_total_extract_df['DR2TPFAT']
individual_total_df['DRTSFAT'] = individual_total_extract_df['DR1TSFAT'] + individual_total_extract_df['DR2TSFAT']
individual_total_df['DRTSODI'] = individual_total_extract_df['DR1TSODI'] + individual_total_extract_df['DR2TSODI']

individual_total_df = individual_total_df.groupby(by='SEQN').sum()

individual_total_df = individual_total_df/2

C:\Users\Gstar\AppData\Local\Temp\ipykernel_156\176251906.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_total_extract_df.fillna(0, inplace=True)


In [41]:
# 根据SEQN合并两份数据
HEI_evaluate_df = pd.merge(left=individual_DGA_standard_df, right=individual_total_df, on='SEQN', how='left')
HEI_evaluate_df.reset_index(drop=False, inplace=True)

In [42]:
# 根据HEI 进行评分
score_in_HEI = pd.DataFrame()
score_in_HEI['SEQN'] = HEI_evaluate_df['SEQN']

In [43]:
### Total Fruits评分
total_fruit_per_kcal = HEI_evaluate_df['F_TOTAL'] * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.8
min = 0
points = 5
for i in total_fruit_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Total Fruits'] = score_list

In [44]:
### Whole Fruits评分
whole_fruit_per_kcal = (HEI_evaluate_df['F_TOTAL'] - HEI_evaluate_df['F_JUICE']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.4
min = 0
points = 5
for i in whole_fruit_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)
score_in_HEI['Whole Fruit'] = score_list

In [45]:
### Total Vegetables评分
total_vegetables_per_kcal = (HEI_evaluate_df['V_TOTAL'] + HEI_evaluate_df['PF_LEGUMES']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 1.1
min = 0
points = 5
for i in total_vegetables_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Total Vegetables'] = score_list

In [46]:
### Greens and Beans评分
greens_and_beans_per_kcal = (HEI_evaluate_df['V_DRKGR'] + HEI_evaluate_df['PF_LEGUMES']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.2
min = 0
points = 5
for i in greens_and_beans_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Greens and Beans'] = score_list

In [47]:
### Whole Grains评分
whole_grains_per_kcal = (HEI_evaluate_df['G_WHOLE']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 1.5
min = 0
points = 10
for i in whole_grains_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Whole Grains'] = score_list

In [48]:
### Dairy评分
dairy_per_kcal = (HEI_evaluate_df['D_TOTAL']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 1.3
min = 0
points = 10
for i in dairy_per_kcal:
    if i >= max:
        score = points  
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Dairy'] = score_list

In [49]:
### Total Protein Foods评分
total_protein_foods_per_kcal = (HEI_evaluate_df['PF_TOTAL']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 2.5
min = 0
points = 5
for i in total_protein_foods_per_kcal:
    if i >= max:
        score = points    
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Total Protein'] = score_list

In [50]:
### Seafood and Plant Proteins评分
seafood_and_plant_proteins_per_kcal = (HEI_evaluate_df['PF_SEAFD_HI'] + HEI_evaluate_df['PF_SEAFD_LOW'] + HEI_evaluate_df['PF_LEGUMES']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.8
min = 0
points = 5
for i in seafood_and_plant_proteins_per_kcal:
    if i >= max:
        score = points    
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Seafood and Plant Proteins'] = score_list

In [51]:
### Fatty Acids评分
fatty_acids = (HEI_evaluate_df['DRTMFAT'] + HEI_evaluate_df['DRTPFAT']) /  HEI_evaluate_df['DRTSFAT']
score_list = []
max = 2.5
min = 1.2
points = 10
for i in fatty_acids:
    if i >= max:
        score = points    
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Fatty Acids'] = score_list

In [52]:
### Refined Grains评分
refined_grains_per_kcal = (HEI_evaluate_df['G_REFINED']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 4.3
min = 1.8
points = 10
for i in refined_grains_per_kcal:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Refined Grains'] = score_list

In [53]:
### Sodium评分
sodium = (HEI_evaluate_df['DRTSODI']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 2.0
min = 1.1
points = 10
for i in sodium:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Sodium'] = score_list

In [54]:
### Added Sugars评分
added_sugars_per_kcal = (HEI_evaluate_df['ADD_SUGARS']) * 16 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.26
min = 0.065
points = 10
for i in added_sugars_per_kcal:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Added Sugars'] = score_list

In [55]:
### Saturated Fats评分
saturated_fats_per_kcal = (HEI_evaluate_df['DRTSFAT']) * 9 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.16
min = 0.08
points = 10
for i in saturated_fats_per_kcal:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Saturated Fats'] = score_list

In [56]:
# 求总分
score_sum_HEI = score_in_HEI
score_sum_HEI['diet_score'] = score_in_HEI.drop(labels='SEQN', axis=1).sum(axis=1)

In [57]:
score_in_HEI

,SEQN,Total Fruits,Whole Fruit,Total Vegetables,Greens and Beans,Whole Grains,Dairy,Total Protein,Seafood and Plant Proteins,Fatty Acids,Refined Grains,Sodium,Added Sugars,Saturated Fats,diet_score
0,41475.0,3.540281,1.846672,5.000000,5.000000,0.000000,10.000000,5.000000,5.000000,0.000000,0.000000,0.0,10.000000,0.879048,46.266000
1,41476.0,5.000000,5.000000,3.533310,5.000000,2.415796,10.000000,4.805382,0.000000,5.676234,10.000000,0.0,5.598531,8.309996,65.339247
2,41477.0,3.260611,5.000000,5.000000,0.000000,0.000000,0.972715,5.000000,5.000000,10.000000,0.000000,0.0,8.632012,8.115578,50.980916
3,41478.0,5.000000,4.762086,2.504823,0.000000,10.000000,10.000000,3.208348,0.000000,0.000000,10.000000,0.0,6.353750,3.174978,55.003986
4,41479.0,3.986209,5.000000,5.000000,0.055193,1.302394,4.385824,5.000000,5.000000,8.914594,0.000000,0.0,6.099030,7.843287,52.586532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9250,51619.0,0.940832,1.881664,5.000000,5.000000,1.649414,6.660540,5.000000,5.000000,2.674476,0.000000,0.0,9.112658,3.519778,46.439361
9251,51620.0,0.000000,0.000000,5.000000,5.000000,7.786688,4.874313,5.000000,5.000000,3.699656,0.000000,0.0,6.311923,9.476371,52.148951
9252,51621.0,5.000000,5.000000,2.172875,0.442023,0.000000,8.346280,5.000000,3.587582,0.000000,4.919408,0.0,3.709740,8.192126,46.370034
9253,51622.0,5.000000,5.000000,1.313578,1.720162,0.082354,10.000000,0.000000,0.000000,0.316980,6.564295,0.0,9.826526,7.426770,47.250665


In [58]:
score_in_HEI.to_excel('E:/work/Paper/NHANES/0708Analysis Data/Final data/DGA_HEI_score.xlsx')